**Challenge context**

Elmy aimerait disposer d’un modèle de prévision permettant de juger en amont si les prix de l’électricité sur le marché SPOT (marché d’enchère européen permettant d’acheter de l’électricité la veille pour le lendemain) seront plus ou moins élevés que les prix de l’électricité sur le marché Intraday (marché boursier européen permettant d’acheter de l’électricité le jour même).

**Challenge goals**

L’exercice consiste en la modélisation supervisée de l'écart de prix entre le marché infra-journalier (dit "Intraday") et le marché SPOT. L'écart de prix peut être modélisé par une régression mais aussi par une classification car ce qui importe avant tout est de prédire correctement le sens de cet écart (si tel ou tel prix sera supérieur ou inférieur à l’autre).

**Data description**
* Index
>* DELIVERY_START : date et heure de livraison de l'électricité

* Variables explicatives
>* load_forecast : prévision de consommation totale d'éléctricité en France
>* coal_power_available, gas_power_available, nucelear_power_available : capacité de production totale d'électricité des centrales à charbon, gaz et nucléaire respectivement,
>* wind_power_forecasts_average, solar_power_forecasts_average : moyenne de différentes prévisions de production totale d'électricité éolienne et solaire (respectivement),
>* wind_power_forecasts_std, solar_power_forecasts_std : écart-type de ces mêmes prévisions,
>* predicted_spot_price : prévision du prix SPOT de l'électricité issues d'un modèle interne de Elmy. Ce modèle est lancé chaque jour avant la fermeture des enchères SPOT pour le lendemain.

* Variable cible
>* spot_id_delta : l'écart entre le VWAP des transactions sur le marché infra-journalier (Intraday) et le prix SPOT pour 1MWh d'électricité (spot_id_delta = Intraday - SPOT) : si la valeur est positive, le prix Intraday est supérieur au prix SPOT et inversement.

**Métrique d'évaluation des performances du modèle**

Puisque l'objectif est avant de tout de prédire correctement le sens de l'écart, on s'appuyera sur une métrique de classification pour évaluer la performance d'un modèle. La grandeur de l'écart observé nous importe aussi : plus l'écart observé est important, plus il est important de prédire le correctement son sens. La métrique de performance proposée pour ce challenge est donc la Weighted Accuracy. C'est à dire la proprotion des predictions dont les sens (positif ou négatif) est correctement identifié pondérée par la valeur absolue des écarts réellement observés

In [34]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns


In [25]:
x_train = pd.read_csv("x_train.csv", index_col= 'DELIVERY_START', parse_dates=True)
y_train = pd.read_csv("y_train.csv", index_col= 'DELIVERY_START', parse_dates=True)

x_test = pd.read_csv("x_test.csv", index_col= 'DELIVERY_START', parse_dates=True)

In [28]:
print(y_train.index.min().date())

2022-01-01


In [29]:
print(f"Training data : from {x_train.index.min().date()} to {x_train.index.max().date()}")
print(f"Test data : from {x_test.index.min().date()} to {x_test.index.max().date()}")

Training data : from 2022-01-01 to 2023-03-29
Test data : from 2023-04-02 to 2023-10-24


In [32]:
x_train.shape

(10605, 9)

In [33]:
x_test.shape

(4942, 9)

In [35]:
sns.heatmap(x_train.isna())

<Axes: ylabel='DELIVERY_START'>